# <p style="text-align: center;">**Bootcamp Big Data & Machine Learning V</p>**

**<p style="text-align: center;">NLP - Práctica Final</p>**</br>
**<p style="text-align: center;">Alberto Muñoz Freán</p>**

**2) Análisis de Sentimiento:**

En segundo lugar, generaremos un modelo de clasificación de binaria capaz de separar las reviews de nuestro dataset de Amazon en dos categorías: buenas (3 a 5 estrellas) o malas (1 o 2 estrellas):

In [20]:
#Importamos todo lo necesario para el análisis de sentimiento:
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve

import matplotlib.pyplot as plt

import gensim
from gensim.corpora import Dictionary
from nltk.stem import WordNetLemmatizer
from gensim.models import LdaModel, CoherenceModel

In [2]:
#Cargamos los archivos json que contienen las reviews en sendos dataframe:
reviews_tools = pd.read_json('Tools_and_Home_Improvement_5.json', orient='columns', lines=True)
reviews_phones = pd.read_json('Cell_Phones_and_Accessories_5.json', orient='columns', lines=True)
reviews_clothing = pd.read_json('Clothing_Shoes_and_Jewelry_5.json', orient='columns', lines=True)
reviews_sports = pd.read_json('Sports_and_Outdoors_5.json', orient='columns', lines=True)

In [3]:
#Unimos los cuatro subsets en un solo data frame:
df1 = reviews_tools.combine_first(reviews_phones)
df2 = df1.combine_first(reviews_clothing)
df3 = df2.combine_first(reviews_sports)

#Hacemos un shuffle de todos los resultados, de modo que estén mezclados y no por orden de entrada en el dataset conjunto:
df_full = df3.sample(frac=1)

#Confirmamos el resultado:
df_full.shape

(296337, 9)

In [4]:
#Del dataframe completo, solo nos quedamos con el texto de la review (reviewText) y las estrellas (overall):
df_final = df_full[["reviewText","overall"]]

#Eliminamos los nulos, incompatibles con el modelo:
df_final.dropna(inplace=True)

#Reiniciamos los índices (cada registro mantenía el índice heredado del primer dataframe, hecho con los archivos json):
df_final.reset_index(drop=True, inplace=True)

#Comprobamos el resultado:
df_final.head()

C:\Users\amfre\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,reviewText,overall
0,Works as expected - it is the five mode versio...,5.0
1,I am kind of disappointed with this item. It w...,3.0
2,Not only was the product just as nice as the p...,5.0
3,The Bolse AON6 battery pack is a basic medium ...,3.0
4,I needed to replace the Siemens switch on my I...,3.0


In [5]:
#Lo siguiente es añadir el "sentimiento" al dataframe mediante etiquetas, de forma que un rating menor a tres indique 
#una review "negativa", y un rating de 3 o más sea una review "positiva":

#Para ello, definimos una función que etiquete cada registro según nuestros criterios:
def label_sentiment(row):
    if int(row['overall']) < 3:
        return 'neg'
    else:
        return 'pos'

#Llamamos a la función sobre una nueva columna del dataframe: "sentiment". Esta columna almacenará todas las etiquetas.
df_final['sentiment'] = df_final.apply(lambda row: label_sentiment(row), axis=1)

#Comprobamos el resultado:
df_final.head()

C:\Users\amfre\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,reviewText,overall,sentiment
0,Works as expected - it is the five mode versio...,5.0,pos
1,I am kind of disappointed with this item. It w...,3.0,pos
2,Not only was the product just as nice as the p...,5.0,pos
3,The Bolse AON6 battery pack is a basic medium ...,3.0,pos
4,I needed to replace the Siemens switch on my I...,3.0,pos


In [7]:
#Con el dataframe ya etiquetado, separamos los datos en train y test:
x_train, x_test, y_train, y_test = train_test_split(
    df_final['reviewText'], # x_train y x_test -> Información a evaluar por el modelo para determinar el sentimiento.
    df_final['sentiment'], # y_train e y_test -> Información con la que determinar si el modelo ha acertado o no en su decisión.
    train_size=0.7, # El 70% de los datos serán de entrenamiento.
    test_size=0.3, # El 30% de los datos serán de test.
    random_state=42, # Fijamos la seed para asegurar la reproducibilidad del proceso.
    shuffle=True # Barajamos los registros antes de separarlos.
)

In [8]:
#Guardamos el resultado en sendos csv para poder reutilizarlos en caso de tener que cerrar el notebook:
x_train.to_csv(r'x_train_SA.csv', index = True)
x_test.to_csv(r'x_test_SA.csv', index = True)
y_train.to_csv(r'y_train_SA.csv', index = True)
y_test.to_csv(r'y_test_SA.csv', index = True)

Una vez separados los datos, el siguiente paso es preprocesarlos y entrenar un modelo para predecir el "sentimiento" de cada review. Dado que buscamos una clasificación binaria (positiva o negativa), usaremos la regresión logística de scikit learn:

**A) Entrenar un modelo sin preprocesado:**

In [14]:
#Para poder trabajar con los datos de x_train, necesitaremos convertir las cadenas de texto en parámetros que 
#un modelo pueda interpretar (una matriz de features TF-IDF). Para ello, haremos una extracción de características:

#Generamos la función para extraer características:
fts = TfidfVectorizer( 
    lowercase=False, #No queremos cambio a minúsculas por defecto.
    ngram_range=(2, 3)) #Extraeremos solo bigramas y trigramas. 

#Ejecutamos el vectorizador sobre el dataset de train.
fts.fit(x_train)

#Una vez tenemos la matriz, obtenemos los scores TF-IDF de la misma, que usaremos para entrenar el modelo. 
x_train_ = fts.transform(x_train)

In [15]:
#Para hacer regresión logística, necesitamos un parámetro de regularización (C): cuanto más pequeño sea, mayor regularización
#aplicará el modelo. Con este grid de parámetros (c_par), podemos observar el comportamiento del modelo en cada situación
#y seleccionar el valor óptimo:
c_par = [0.01, 0.1, 1, 10]

#Creamos una lista vacía en la que se guardarán los resultados de Accuracy del modelo sobre el dataset de train:
train_acc = list()

#Creamos una función que aplicará una regresión logística sobre el dataset por cada parámetro de C:
for c in c_par:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500) #lbfgs es un buen algoritmo para resolver problemas multiclase.
    lr.fit(x_train_, y_train)
    
    #Se hará una predicción de los resultados de train:
    train_predict = lr.predict(x_train_)
    
    #Imprimimos el resultado:
    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_train, train_predict)))
    
    #Añadimos a la lista vacía train_acc los resultados de Accuracy obtenidos por el modelo:
    train_acc.append(accuracy_score(y_train, train_predict))

Accuracy for C=0.01: 0.9109166726926508
Accuracy for C=0.1: 0.9109263142671198
Accuracy for C=1: 0.9267529587581652
Accuracy for C=10: 0.9994697134041989


**B) Entrenar un modelo preprocesado:**

En este caso, someteremos al modelo a un preprocesamiento muy similar al utilizado en topic modeling:<br>
<br>1: Conversión de mayúsculas a minúsculas<br>
2: Eliminación de acentos y signos de puntuación<br>
3: Eliminación de palabras de menos de 3 caracteres<br>
4: Eliminación de stopwords

In [12]:
#Utilizaremos el preprocesador de Topic Modeling para tokenizar, eliminar stopwords y palabras de menos de 3 caracteres:
def text_preprocessing(text):
    result=[] 
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3: 
            result.append(token)
    return result

#Generamos la función TfidfVectorizer para extraer características:
fts = TfidfVectorizer( 
    strip_accents='ascii', #Eliminamos los acentos y signos de puntuación con ASCII.
    lowercase=True, #Convertimos todas las palabras a minúsculas.
    tokenizer=text_preprocessing, #Llamamos a la función text_preprocessing para que tokenice y preprocese.
    ngram_range=(2, 3)) #Extraeremos solo bigramas y trigramas.
    
#Ejecutamos el vectorizador sobre el dataset de train.
fts.fit(x_train)

#Una vez tenemos la matriz, obtenemos los scores TF-IDF de la misma, que usaremos para entrenar el modelo. 
x_train_2 = fts.transform(x_train)


C:\Users\amfre\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [13]:
#Entrenamos el modelo de la misma forma que en el caso A:
c_par = [0.01, 0.1, 1, 10]

train_acc = list()

for c in c_par:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500)
    lr.fit(x_train_2, y_train)
    
    train_predict = lr.predict(x_train_2)
    
    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_train, train_predict)))
    
    train_acc.append(accuracy_score(y_train, train_predict))

Accuracy for C=0.01: 0.9109166726926508
Accuracy for C=0.1: 0.9109166726926508
Accuracy for C=1: 0.9135825680333598
Accuracy for C=10: 0.9994841757659025


**C) Modelo preprocesado y lemmatizado:**

In [54]:
#Generamos una función a partir de text_processing que sea capaz de lemmatizar:
def text_pl(text):
    result=[] 
    lemmatizer = WordNetLemmatizer()
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3: 
            result.append(token)
    return [lemmatizer.lemmatize(w) for w in result]

#Usamos TfidfVectorizer para extraer características:
fts = TfidfVectorizer( 
    strip_accents='ascii', #Eliminamos los acentos y signos de puntuación con ASCII.
    lowercase=True, #Convertimos todas las palabras a minúsculas.
    tokenizer=text_pl, #Llamamos a la función text_pl para que tokenice, preprocese y lemmatice.
    ngram_range=(2, 3)) #Extraeremos solo bigramas y trigramas.
    
#Ejecutamos el vectorizador sobre el dataset de train.
fts.fit(x_train)

#Una vez tenemos la matriz, obtenemos los scores TF-IDF de la misma, que usaremos para entrenar el modelo. 
x_train_2 = fts.transform(x_train)

C:\Users\amfre\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [55]:
#Entrenamos el modelo de la misma forma que en los casos A y B:
c_par = [0.01, 0.1, 1, 10]

train_acc = list()

for c in c_par:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500)
    lr.fit(x_train_2, y_train)
    
    train_predict = lr.predict(x_train_2)
    
    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_train, train_predict)))
    
    train_acc.append(accuracy_score(y_train, train_predict))

Accuracy for C=0.01: 0.9109166726926508
Accuracy for C=0.1: 0.9109166726926508
Accuracy for C=1: 0.913765757948273
Accuracy for C=10: 0.999479354978668


En general, el preprocesamiento no parece haber alterado mucho los valores de Accuracy. Esto, unido a que la Accuracy en casos de baja regularización es de prácticamente el 100%, nos hace pensar que los datos pueden sufrir cierto overfitting, sobre todo si los valores de C no son suficientemente bajos.<br>

Teniendo en cuenta estos resultados, podríamos quedarnos con el modelo sin procesar por simplicidad, pero considero que es mejor preprocesar todo lo posible, pues facilitamos al modelo el trabajo en todo lo posible, y con ello mejoraremos la Accuracy en caso de que test tenga una performance más pobre, lo cual es probable. Por tanto, repetiremos el preprocesamiento del **caso C**:

In [56]:
#Usamos TfidfVectorizer para extraer características:
fts = TfidfVectorizer( 
    strip_accents='ascii', #Eliminamos los acentos y signos de puntuación con ASCII.
    lowercase=True, #Convertimos todas las palabras a minúsculas.
    tokenizer=text_pl, #Llamamos a la función text_pl para que tokenice, preprocese y lemmatice.
    ngram_range=(2, 3)) #Extraeremos solo bigramas y trigramas.
    
#Ejecutamos el vectorizador sobre el dataset de test.
fts.fit(x_test)

#Una vez tenemos la matriz, obtenemos los scores TF-IDF de la misma, que usaremos para entrenar el modelo. 
x_test_2 = fts.transform(x_test)

C:\Users\amfre\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [57]:
#Observamos cómo funciona el modelo sobre los datos de test:
c_par = [0.01, 0.1, 1, 10] 

test_acc = list()

for c in c_par:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500)
    lr.fit(x_test_2, y_test)
    
    test_predict = lr.predict(x_test_2)
    
    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_test, test_predict)))
    
    test_acc.append(accuracy_score(y_test, test_predict))

Accuracy for C=0.01: 0.9126453848057412
Accuracy for C=0.1: 0.9126453848057412
Accuracy for C=1: 0.9134327686666217
Accuracy for C=10: 0.9997412881599964


La Accuracy de los datos de test es muy similar a la de los datos de train, lo cual indicaría que el modelo es bastante eficaz a la hora de determinar si las reviews son positivas o negativas, y descartaría a priori la posibilidad de overfitting.<br>

La limitación principal de este modelo es que no tiene en cuenta las ambigüedades o neutralidades, es decir, las reviews de 3 estrellas, que en principio indican que el producto tiene cosas buenas y malas. Una manera de mejorar las prestaciones del modelo sería añadir una tercera etiqueta (neutral) y observar entonces cómo se comporta.<br>

También podríamos explorar otros modelos de clasificación para análisis de sentimiento, pues existen varios [papers](https://paperswithcode.com/task/sentiment-analysis) y [artículos](https://towardsdatascience.com/sentiment-analysis-for-text-with-deep-learning-2f0a0c6472b5) que cuentan con modelos ya preparados y entrenados que podrían ser más eficaces, o incluso permitir clasificaciones más complejas que se nos pudieran ocurrir (por ejemplo, una etiqueta para cada rating/estrella).